In [1]:
# Author: lyh 
# Date  : 2020-09-19
# 使用了分布式学习的ImageNet训练代码
# 使用以下命令直接执行
# CUDA_VISIBLE_DEVICES=0,1,2,3,4,5,6 python -m torch.distributed.launch --nproc_per_node=7 example_ES_res18.py
from __future__ import print_function
import sys
sys.path.append("..")
from datasets.es_imagenet import ESImagenet_Dataset
import LIAF
from LIAFnet.LIAFResNet import *
import torch.distributed as dist 
import torch.nn as nn
import argparse, pickle, torch, time, os,sys
from importlib import import_module


##################### Step1. Env Preparation #####################



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


##################### Step2. load in dataset #####################

modules = import_module('LIAFnet.LIAFResNet_18')
config  = modules.Config()
workpath = os.path.abspath(os.getcwd())

num_epochs = config.num_epochs
batch_size = config.batch_size
timeWindows = config.timeWindows
epoch = 0
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
training_iter = 0
start_epoch = 0
acc_record = list([])
loss_train_record = list([])
loss_test_record = list([])

batch_size = 18 * 4

test_dataset = ESImagenet_Dataset(mode='test',data_set_path='/data/ES-imagenet-0.18/')
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=8,pin_memory=True)

##################### Step3. establish module #####################

snn = LIAFResNet(config)
print("Total number of paramerters in networks is {}  ".format(sum(x.numel() for x in snn.parameters())))

#state_dict = torch.load(save_folder+'/'+str(start_epoch)+'modelsaved.t7')
#print(state_dict['state'])
#snn.load_state_dict(state_dict['state'])

snn = LIAFResNet(config)
snn=torch.nn.SyncBatchNorm.convert_sync_batchnorm(snn)
snn.to(device)

##########################################################
# 修改部分3
# 载入模型
##########################################################
print('using uniformed init')
pretrain_path = '../pretrained_model/ResNet18-Acc52.pkl'
checkpoint = torch.load(pretrain_path, map_location=torch.device('cpu'))

#print(checkpoint)
snn.load_state_dict(checkpoint)


################step4. training and validation ################

def val(snn,test_loader,test_dataset,batch_size,epoch):
    snn.eval()
    correct = 0
    total = 0
    predicted = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            if ((batch_idx+1)<=len(test_dataset)//batch_size):
                try:
                    targets=targets.view(batch_size)#tiny bug
                    outputs = snn(inputs.type(LIAF.dtype))
                    _ , predicted = outputs.cpu().max(1)
                    total += float(targets.size(0))
                    correct += float(predicted.eq(targets).sum())
                    print(batch_idx,'/',len(test_dataset)/batch_size)
                except:
                    print('sth. wrong')
                    print('val_error:',batch_idx, end='')
                    print('taret_size:',targets.size())
    acc = 100. * float(correct) / float(total)
    return acc


acc = val(snn,test_loader,test_dataset,batch_size,epoch=-1)
print('acc:',acc)




Total number of paramerters in networks is 11693233  
using uniformed init


/home/lyh/anaconda3/envs/pytorch/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448255797/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


0 / 692.7916666666666
1 / 692.7916666666666
2 / 692.7916666666666
3 / 692.7916666666666
4 / 692.7916666666666
5 / 692.7916666666666
6 / 692.7916666666666
7 / 692.7916666666666
8 / 692.7916666666666
9 / 692.7916666666666
10 / 692.7916666666666
11 / 692.7916666666666
12 / 692.7916666666666
13 / 692.7916666666666
14 / 692.7916666666666
15 / 692.7916666666666
16 / 692.7916666666666
17 / 692.7916666666666
18 / 692.7916666666666
19 / 692.7916666666666
20 / 692.7916666666666
21 / 692.7916666666666
22 / 692.7916666666666
23 / 692.7916666666666
24 / 692.7916666666666
25 / 692.7916666666666
26 / 692.7916666666666
27 / 692.7916666666666
28 / 692.7916666666666
29 / 692.7916666666666
30 / 692.7916666666666
31 / 692.7916666666666
32 / 692.7916666666666
33 / 692.7916666666666
34 / 692.7916666666666
35 / 692.7916666666666
36 / 692.7916666666666
37 / 692.7916666666666
38 / 692.7916666666666
39 / 692.7916666666666
40 / 692.7916666666666
41 / 692.7916666666666
42 / 692.7916666666666
43 / 692.791666666666

346 / 692.7916666666666
347 / 692.7916666666666
348 / 692.7916666666666
349 / 692.7916666666666
350 / 692.7916666666666
351 / 692.7916666666666
352 / 692.7916666666666
353 / 692.7916666666666
354 / 692.7916666666666
355 / 692.7916666666666
356 / 692.7916666666666
357 / 692.7916666666666
358 / 692.7916666666666
359 / 692.7916666666666
360 / 692.7916666666666
361 / 692.7916666666666
362 / 692.7916666666666
363 / 692.7916666666666
364 / 692.7916666666666
365 / 692.7916666666666
366 / 692.7916666666666
367 / 692.7916666666666
368 / 692.7916666666666
369 / 692.7916666666666
370 / 692.7916666666666
371 / 692.7916666666666
372 / 692.7916666666666
373 / 692.7916666666666
374 / 692.7916666666666
375 / 692.7916666666666
376 / 692.7916666666666
377 / 692.7916666666666
378 / 692.7916666666666
379 / 692.7916666666666
380 / 692.7916666666666
381 / 692.7916666666666
382 / 692.7916666666666
383 / 692.7916666666666
384 / 692.7916666666666
385 / 692.7916666666666
386 / 692.7916666666666
387 / 692.791666

688 / 692.7916666666666
689 / 692.7916666666666
690 / 692.7916666666666
691 / 692.7916666666666
acc: 52.29206807964034


In [2]:
print('acc:',acc)

acc: 52.29206807964034
